# BPL's AI4Good Wheat Price Shock Forecasting
Predict wheat prices after the Ukraine invasion disrupted global food markets. Created by the Better Planet Lab for the AI for Good course.
* Kaggle competition here: https://www.kaggle.com/competitions/bpl-ai4good-wheat-price-forecasting
* Resource on time series here: https://medium.com/@ycwong.joe/a-brief-history-of-time-series-models-38455c2cd78d

![header](https://www.kaggle.com/competitions/95742/images/header)





## Setup Environment

In [25]:
!pip install mapclassify --quiet

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd # 
import folium
import matplotlib
import mapclassify

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bpl-ai4good-wheat-price-forecasting/sample_submission.csv
/kaggle/input/bpl-ai4good-wheat-price-forecasting/train.csv
/kaggle/input/bpl-ai4good-wheat-price-forecasting/test.csv


## Data Exploration

In [2]:
data_path = '/kaggle/input/bpl-ai4good-wheat-price-forecasting/'
sample_submission_filename = 'sample_submission.csv'
train_data = 'train.csv'
test_data = 'test.csv'

In [33]:
df = pd.read_csv(f'{data_path}{train_data}')
df = df.dropna()
df

,admin0,admin1,admin2,market,latitude,longitude,commodity,unit,currency,year,month,date,price_localcurrency,price_usd,igc_wheat,igc_maize,igc_rice,igc_barley,market_year_month
0,Afghanistan,Badakhshan,Argo,Badakhshan,37.037850,70.456210,Wheat,KG,AFN,2020,5,May 2020,31.00,0.3537,187.262857,163.365714,189.701905,188.883333,Badakhshan_2020_5
1,Afghanistan,Badakhshan,Argo,Badakhshan,37.037850,70.456210,Wheat,KG,AFN,2020,6,Jun 2020,27.53,0.3142,181.425797,169.975637,188.884586,186.173100,Badakhshan_2020_6
2,Afghanistan,Badakhshan,Argo,Badakhshan,37.037850,70.456210,Wheat,KG,AFN,2020,7,Jul 2020,28.23,0.3221,182.006258,175.961598,183.553514,186.774202,Badakhshan_2020_7
3,Afghanistan,Badakhshan,Argo,Badakhshan,37.037850,70.456210,Wheat,KG,AFN,2020,8,Aug 2020,26.84,0.3062,183.747500,180.537500,186.884000,192.984500,Badakhshan_2020_8
4,Afghanistan,Badakhshan,Argo,Badakhshan,37.037850,70.456210,Wheat,KG,AFN,2020,9,Sep 2020,26.22,0.2992,194.529291,198.127633,187.856328,200.902690,Badakhshan_2020_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90934,Zimbabwe,Matabeleland North,Nkayi,Nkayi Growth Point,-18.999158,28.898316,Wheat,KG,USD,2015,9,Sep 2015,0.46,0.4600,170.095888,175.636799,150.093624,179.841218,Nkayi Growth Point_2015_9
90935,Zimbabwe,Matabeleland North,Nkayi,Nkayi Growth Point,-18.999158,28.898316,Wheat,KG,USD,2015,10,Oct 2015,0.46,0.4600,175.377449,177.351163,151.367839,188.015136,Nkayi Growth Point_2015_10
90936,Zimbabwe,Matabeleland North,Nkayi,Nkayi Growth Point,-18.999158,28.898316,Wheat,KG,USD,2015,11,Nov 2015,0.46,0.4600,171.890650,177.157004,151.931130,182.937215,Nkayi Growth Point_2015_11
90937,Zimbabwe,Matabeleland North,Nkayi,Nkayi Growth Point,-18.999158,28.898316,Wheat,KG,USD,2015,12,Dec 2015,0.46,0.4600,168.279336,176.679063,150.874879,181.703535,Nkayi Growth Point_2015_12


In [38]:
markets = df[['market', 'latitude', 'longitude']]
markets = markets.drop_duplicates()
markets

,market,latitude,longitude
0,Badakhshan,37.037850,70.456210
46,Badghis,35.136990,63.734460
92,Baghlan,35.804450,68.914200
138,Balkh,36.669910,67.078410
184,Bamyan,34.824167,67.821667
...,...,...,...
90450,Soqatra (Hudaibo),12.650000,54.033333
90634,Taiz City,13.580000,44.020000
90808,Zungubar City,13.129193,45.377636
90905,Mbare,-17.858056,31.037222


In [37]:
gdf = gpd.GeoDataFrame(markets, geometry=gpd.points_from_xy(markets.longitude, markets.latitude), crs="EPSG:4326")

gdf.explore()

## Assignment
Win the competition! Details here: https://www.kaggle.com/competitions/bpl-ai4good-wheat-price-forecasting. After submitting, share a link to your notebook along with your metric in Discord.

# 